In [2]:
gurobi_cut = np.zeros((100, 2000), dtype=int)
import json

data = {}
ser = []
with open("cut_result") as f:
    for i in f.readlines():
        i = i.replace("inf", "1e100")
        d = eval(i)
        for n, seed, lb, ub in d:
            # assert abs(lb - ub) < 1e-6
            gurobi_cut[n, seed] = ub
            if n not in data:
                data[n] = {}
            data[n][seed] = {"upper bound": ub}
            # assert abs(gurobi_cut[n, seed] - ub) < 1e-6

# json.dump(data, open("data/cut_upperbounds.json", "w"))

NameError: name 'np' is not defined

In [ ]:
import itertools


def read_chain(x):
    with open(x, "r") as f:
        return list(itertools.chain.from_iterable(map(eval, f.readlines())))


def work(file, entry):
    raw = read_chain(file)
    a, b = entry.split("-")
    for n, seed, _, _, ans in raw:
        if a not in data[n][seed]:
            data[n][seed][a] = {}
        data[n][seed][a][b] = ans


work("approx0.95", "3/4-0.95")
work("approx1", "3/4-1")

work("approx230.95", "2/3-0.95")
work("approx231", "2/3-1")

json.dump(data, open("data/scaling.json", "w"))

In [1]:
import json
import numpy as np

data = json.load(open("data/scaling.json"))

from rsmf import CustomFormatter

formatter = CustomFormatter(
    columnwidth=390 * 0.01389,
    wide_columnwidth=542.02501 * 0.01389,
    fontsizes=12,
    pgf_preamble=r"""\usepackage{mathptmx}
    \usepackage{charter}
    \renewcommand{\familydefault}{\sfdefault}""",
)
import numpy as np
from matplotlib import pyplot as plt
from rsmf import CustomFormatter


plt.rcParams["font.family"] = "serif"
plt.rcParams["text.usetex"] = True
# )
fig = formatter.figure()

In [2]:
print(plt.style.available)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [3]:
# plt.style.use('ggplot')
for setting in ["3/4", "2/3"]:
    formatter.figure(aspect_ratio=0.8, width_ratio=0.49)
    plt.cla()

    import matplotlib.patches as mpatches

    labels = []

    def add_label(violin, label):
        color = violin["bodies"][0].get_facecolor().flatten()
        labels.append((mpatches.Patch(color=color), label))

    ns = sorted([int(n) for n in data if setting in data[n]["1"]])

    for idx, r in enumerate([1, 0.95]):
        ax = plt.gca()
        # ns = list(range(8, 41, 4))
        ratio = []

        for n in ns:
            ls = []
            for i in range(1, 1000 + 1):
                entry = data[str(n)][str(i)]
                ls.append(entry[setting][str(r)] / entry["upper bound"])
            ratio.append(ls)

        plots = ax.violinplot(ratio, showextrema=True, showmeans=True, widths=0.5)

        add_label(plots, rf"$\alpha={r}$")
        color = f"C{idx}"

        for pc in plots["bodies"]:
            pc.set_facecolor(color)
            # pc.set_edgecolor(color)
            # pc.set_alpha(0.6)
        for partname in ("cbars", "cmins", "cmaxes"):
            pc = plots[partname]
            pc.set_edgecolor(color)
            pc.set_linewidth(0.2)
        plots["cmeans"].set_edgecolor(color)
        plots["cmeans"].set_linewidth(2)

        xlim = plt.xlim()
        plt.plot(xlim, [r, r], "k--", linewidth=0.2, color=color)

        plt.xlim(xlim)

        plt.xticks(range(1, len(ns) + 1), ns)
        plt.xlabel("$n$")
        plt.ylabel("approximation ratio")
        plt.tight_layout()

    plt.ylim(0.92 - 0.005, 1.005)
    plt.yticks(np.arange(0.92, 1.01, 0.02))
    plt.tight_layout()
    plt.legend(*zip(*labels), loc="lower right")
    plt.savefig(
        "fig/" + f"approx{setting}.pdf".replace("/", ""),
        bbox_inches="tight",
        pad_inches=0,
    )

/tmp/ipykernel_14715/1728207776.py:45: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k--" (-> color='k'). The keyword argument will take precedence.
  plt.plot(xlim, [r, r], "k--", linewidth=0.2, color=color)
/tmp/ipykernel_14715/1728207776.py:45: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k--" (-> color='k'). The keyword argument will take precedence.
  plt.plot(xlim, [r, r], "k--", linewidth=0.2, color=color)
/tmp/ipykernel_14715/1728207776.py:45: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k--" (-> color='k'). The keyword argument will take precedence.
  plt.plot(xlim, [r, r], "k--", linewidth=0.2, color=color)


In [ ]:
plt.cla()
ratio = np.array(ratio)
plt.plot(ns, ratio.mean(axis=1), label=r"$\alpha=1")
# plt.fill_between(
#     ns,
#     ratio.min(axis=1),
#     ratio.max(axis=1),
#     alpha=0.3,
# )

plt.fill_between(
    ns,
    ratio.mean(axis=1) - ratio.std(axis=1),
    ratio.mean(axis=1) + ratio.std(axis=1),
    alpha=0.3,
)

for i in ratio:
    assert max(i) <= 1

plt.xlabel("$n$")
plt.ylabel("approximation ratio")
plt.legend()
plt.tight_layout()
plt.savefig("approx1_mean.pdf", bbox_inches="tight", pad_inches=0)

In [ ]:
def set_seed(seed):
    import random as rnd
    import numpy as np

    np.random.seed(seed)
    rnd.seed(seed)


set_seed(2)
import networkx as nx

nx.erdos_renyi_graph(n=8, p=0.8)

In [ ]:
scaling = {}

import itertools


def read_chain(x):
    with open(x, "r") as f:
        return list(itertools.chain.from_iterable(map(eval, f.readlines())))


scaling["3/4"] = {0.95: read_chain("approx0.95")}
scaling["3/4"][1] = read_chain("approx1")

scaling["2/3"] = {0.95: read_chain("approx230.95")}
scaling["2/3"][1] = read_chain("approx231")
scaling
import json

json.dump(scaling, open("data/scaling.json", "w"))